<a href="https://colab.research.google.com/github/saisrigundam/GB_730_Course_Assigments/blob/main/P2_Gundam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set-Up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo.
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

In [ ]:
import pandas as pd

#Code that solves one instance of the Markdown Pricing Problem

In [ ]:
 #reading in markdown pricing data and previewing the data
markdown_pricing_data = pd.read_excel('MP_scenarios.xlsx', sheet_name='Sheet1')
markdown_pricing_data

,No,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value
0,1,55.48,120,160,194,223,15,1421,13.870
1,2,53.68,115,149,171,197,15,2396,13.420
2,3,61.56,140,191,207,459,15,2544,15.390
3,4,65.72,115,151,242,278,15,1316,16.430
4,5,64.98,120,173,221,335,15,1377,16.245
...,...,...,...,...,...,...,...,...,...
195,196,58.18,105,152,160,454,15,2320,14.545
196,197,58.58,125,169,177,298,15,1465,14.645
197,198,50.18,145,218,263,362,15,1433,12.545
198,199,59.58,140,185,224,292,15,1355,14.895


In [ ]:
#inputs
#converting each of the column data to a list of lists
list_price = markdown_pricing_data.loc[:, 'Price'].values.tolist()
demand = markdown_pricing_data.loc[:, 'Demand (0% discount)':'Demand (50% discount)'].values.tolist()
weeksleft = markdown_pricing_data.loc[:, 'Weeks Left'].values.tolist()
inv = markdown_pricing_data.loc[:, 'Inventory Remaining'].values.tolist()
sal = markdown_pricing_data.loc[:, 'Salvage Value'].values.tolist()

In [ ]:
#calculate discounted prices
#discount = [0,.15,.30,.50]
#for j in range(num_products):
#initialize the decision variables
#model = ConcreteModel()
#model.x = Var(range(num_products),range(num_DVs),domain=NonNegativeReals)
#model.pprint()
#oBJECTIVE
#model.Objective = Objective(expr = sum(demand[j][i]*model.x[j,i]*(price[j][i]-sal[j]) for i in range(num_DVs) + sal[j]*inv[j]) , sense=maximize)
#model.InvConstraint = Constraint(expr = sum(demand[j][i]*model.x[j,i] for i in range(num_DVs))<=inv[j])
#model.WeeksConstraint = Constraint(expr = sum(model.x[j,i] for i in range(4)) <= weeks_left[j])
#solving
#opt = SolverFactory('glpk')
#opt.solve(model, tee=True)
#assignment_matrix.append([model.x[j,i]() for i in range(num_DVs)])
#projected_revenue_matrix.append(model.Objective())
#model.pprint()

In [ ]:
# function to solve for the opitmal pricing plan and projected revenue
def optimal_pricing_plan_revenue(list_price, demand, weeksleft, inv, sal) :
  #calculate discounted prices
  price = [(list_price), (list_price) * (1-.15), (list_price)* (1-.3), (list_price) *(1-.5)] #for 0%, 15%, 30%, 50% discounts, respectively

  #defining the optimization model
  model = ConcreteModel()

  #DVs
  model.x = Var (range (4), domain=NonNegativeReals)

  #Objective maximise the projected revenue
  model.objective = Objective (expr = sum(demand[i]*model.x[i]*(price[i]-sal) for i in range(4)) + sal*inv, sense=maximize)

  #Constraint to check the inventory and the number of weeks left
  model. InConstraint = Constraint (expr = sum(demand[i]* model.x[i] for i in range (4)) <= inv)
  model.WeeksConstraint = Constraint (expr = sum(model.x[i] for i in range(4)) <= weeksleft)

  #solve the model
  opt = SolverFactory('glpk')
  opt. solve (model, tee=True) #can set tee=True if you want to see the details.

  # return the values for optimal pricing plan and also the projected rvenue
  return((model.x[0](), model.x[1](), model.x[2](), model.x[3]()),model.objective())

In [ ]:
# Using a FOR loop we do the calculation of for optimal pricing plan and projected revenue for each of the 200 products by calling the function
solutions = []
for i in range(len(markdown_pricing_data)):
  markdown_dat = markdown_pricing_data.iloc[i].values.tolist()
  list_price = markdown_dat[1]
  demand = markdown_dat[2:6]
  weeksleft = markdown_dat[6]
  inv = markdown_dat[7]
  sal = markdown_dat[8]
  #calling the function and storing the results in solutions
  solutions.append(optimal_pricing_plan_revenue(list_price, demand, weeksleft, inv, sal))

Streaming output truncated to the last 5000 lines.
Size of triangular part is 2
*     0: obj =   2.945625000e+04 inf =   0.000e+00 (4)
*     2: obj =   1.153867439e+05 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40412 bytes)
Writing basic solution to '/tmp/tmprx0vbvzh.glpk.raw'...
17 lines were written
GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpz2cs3d3y.glpk.raw --wglp /tmp/tmpg2p7o7ib.glpk.glp --cpxlp
 /tmp/tmppqbj8etx.pyomo.lp
Reading problem data from '/tmp/tmppqbj8etx.pyomo.lp'...
3 rows, 5 columns, 9 non-zeros
35 lines were read
Writing problem data to '/tmp/tmpg2p7o7ib.glpk.glp'...
28 lines were written
GLPK Simplex Optimizer, v4.65
3 rows, 5 columns, 9 non-zeros
Preprocessing...
2 rows, 4 columns, 8 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  3.870e+02  ratio =  3.870e+02
GM: min|aij| =  7.302e-01  max|aij| =  1.370e+00  ratio =  1.876e+00
EQ: min|aij| =  5.331e-01 

In [ ]:
#Print out the values to check if the function is returning true values
solutions

[((11.8416666666667, 0.0, 0.0, 0.0), 78837.08000000016),
 ((0.0, 15.0, 0.0, 0.0), 104139.20000000001),
 ((6.29411764705882, 8.70588235294118, 0.0, 0.0), 141254.12752941175),
 ((11.4434782608696, 0.0, 0.0, 0.0), 86487.5200000002),
 ((11.475, 0.0, 0.0, 0.0), 89477.46),
 ((11.0, 0.0, 0.0, 0.0), 61105.0),
 ((8.58518518518519, 0.0, 0.0, 0.0), 80712.76000000004),
 ((13.469387755102, 1.53061224489795, 0.0, 0.0), 115386.74387755073),
 ((13.2727272727273, 0.0, 0.0, 0.0), 81322.00000000015),
 ((9.304, 0.0, 0.0, 0.0), 78642.06),
 ((15.0, 0.0, 0.0, 0.0), 138697.90999999997),
 ((11.4074074074074, 3.59259259259259, 0.0, 0.0), 90660.33155555552),
 ((12.55625, 0.0, 2.44375, 0.0), 114913.79725000002),
 ((14.1080586080586, 0.0, 0.0, 0.891941391941392), 115802.935054945),
 ((0.0, 15.0, 0.0, 0.0), 149400.70500000002),
 ((8.7037037037037, 0.0, 0.0, 0.0), 68925.49999999997),
 ((14.7263157894737, 0.0, 0.273684210526317, 0.0), 87880.7484210527),
 ((7.29655172413793, 0.0, 0.0, 0.0), 57005.04),
 ((6.35971223021

In [ ]:
#creat a new dataframe with solutions
markdown_pricing = pd.DataFrame(solutions)
#Update the column names in the dataframe
markdown_pricing.columns= ["Optimal Pricing Plan", "Projected Revenue"]
#Concatnate this new dataframe to the existing dataframe
markdown_pricing_final=pd.concat([markdown_pricing_data,markdown_pricing], axis=1, join='inner')# joinned the new dataframe with the original dataframe
#Preview the dataframe
markdown_pricing_final

,No,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value,Optimal Pricing Plan,Projected Revenue
0,1,55.48,120,160,194,223,15,1421,13.870,"(11.8416666666667, 0.0, 0.0, 0.0)",78837.080000
1,2,53.68,115,149,171,197,15,2396,13.420,"(0.0, 15.0, 0.0, 0.0)",104139.200000
2,3,61.56,140,191,207,459,15,2544,15.390,"(6.29411764705882, 8.70588235294118, 0.0, 0.0)",141254.127529
3,4,65.72,115,151,242,278,15,1316,16.430,"(11.4434782608696, 0.0, 0.0, 0.0)",86487.520000
4,5,64.98,120,173,221,335,15,1377,16.245,"(11.475, 0.0, 0.0, 0.0)",89477.460000
...,...,...,...,...,...,...,...,...,...,...,...
195,196,58.18,105,152,160,454,15,2320,14.545,"(0.0, 14.8675496688742, 0.0, 0.132450331125827)",113506.482914
196,197,58.58,125,169,177,298,15,1465,14.645,"(11.72, 0.0, 0.0, 0.0)",85819.700000
197,198,50.18,145,218,263,362,15,1433,12.545,"(9.88275862068966, 0.0, 0.0, 0.0)",71907.940000
198,199,59.58,140,185,224,292,15,1355,14.895,"(9.67857142857143, 0.0, 0.0, 0.0)",80730.900000


In [ ]:
#Outputs a single .csv file that adds 2 columns to the information in MP_scenarios.xlsx file:
markdown_pricing_final.to_csv('MP_scenarios_solved.csv', index=False)